In [1]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
import glob
import os
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
date_envio = '2022-04-20'
monto_me_mail = 148638292.13
comision_swap_mail = 146525.55
iva_swap_mail = 23444.09
monto_fondear_mail = 148808261.77
date_envio = pd.to_datetime(date_envio)

# READING FILES

## SWAP

In [3]:
bin1 = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\DISPERSIONES\Catalogo_Bancos.csv'
                  ,dtype={'BIN':object,'Clave de la institución':object,'Nombre de la institución':object})

In [4]:
query_1 = glob.glob(r'G:\My Drive\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\04. APRIL\SWAP 20*.csv', recursive=False)
query_1

['G:\\My Drive\\3.-Settlement\\5.-HISTORICO FILES PAGOS\\01. TRADICIONAL\\2022\\04. APRIL\\SWAP 2022-04-20.csv']

In [5]:
query = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                   .assign(file_name=os.path.basename(f)) for f in query_1])                     
query['CLABE'] = query['bank_account_number'].str[1:]
query['BIN'] = query['bank_account_number'].str[1:4].replace(' ', '',regex=True)
query['date_disbursement'] = date_envio

In [6]:
query['CLABE'] = query['bank_account_number'].str[1:]
query['BIN'] = query['bank_account_number'].str[1:4].replace(' ', '',regex=True)
query['date_disbursement'] = date_envio

In [7]:
query_final = query.merge(bin1[['BIN','Clave de la institución','Nombre de la institución']] ,on='BIN', how='left')
query_final.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,status,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,disbursement_method,swap,file_name,CLABE,BIN,date_disbursement,Clave de la institución,Nombre de la institución
0,00008884-ca93-4758-8aa7-b10f57145850,Dra Abryl,maah25@hotmail.com,VN76SZYZ3,2022-04-20 00:00:00,"1,149.89",PENDING,SANTANDER,2022-04-20 08:15:57,Abryl Mariana Avitia Herrera,'014078567364065679,2022-04-20 08:03:18,SWAP,"Abryl Mariana Avitia Herrera,,1149.89,01407856...",SWAP 2022-04-20.csv,014078567364065679,014,2022-04-20,40014,SANTANDER
1,0000b65a-c2a2-4993-8f4c-05e5b7cbd7c0,Ruvalcaba Peleteros S.A. de C.V.,ruvalcabapeleteros@hotmail.com,VJDSB5MVH,2022-04-20 00:00:00,"1,041.41",PENDING,"Banco Mercantil del Norte, S.A.",2022-04-20 08:16:28,Ruvalcaba Peleteros SA de CV,'072225010863381552,2022-04-20 08:03:18,SWAP,"Ruvalcaba Peleteros SA de CV,,1041.41,07222501...",SWAP 2022-04-20.csv,072225010863381552,072,2022-04-20,40072,BANORTE/IXE
2,0000d758-84b4-4cb0-a165-4f0d3c44864d,MAHARISHI PANTJALI SA DE CV,maharishiclip@gmail.com,78KN95CWP,2022-04-20 00:00:00,"26,631.71",PENDING,"Banco Regional de Monterrey, S.A.",2022-04-20 08:16:28,MAHARISHI PANTJALI S.A. DE C.V.,'058375000003160831,2022-04-20 08:03:18,SWAP,"MAHARISHI PANTJALI S.A. DE C.V.,,26631.71,0583...",SWAP 2022-04-20.csv,058375000003160831,058,2022-04-20,40058,BANREGIO
3,0000e953-6b2c-44d0-a798-9dac1abf24ea,Super Kandy,yanellysegovia18@gmail.com,JZM3R2ZN5,2022-04-20 00:00:00,755.76,PENDING,"Banco Nacional de México, S.A.",2022-04-20 08:15:57,Ana Yanelly Puente Gutierrez,'002580701334955664,2022-04-20 08:03:18,SWAP,"Ana Yanelly Puente Gutierrez,,755.76,002580701...",SWAP 2022-04-20.csv,002580701334955664,002,2022-04-20,40002,BANAMEX
4,00016126-0052-402d-ba8a-95e5e1a1c673,Seguridad Industrial OM,jocabetolan@gmail.com,ZTJJXKZRG,2022-04-20 00:00:00,861.46,PENDING,"Banco Nacional de México, S.A.",2022-04-20 08:15:25,CYNTHIA JOCABET OLAN RODRIGUEZ,'002045701331339825,2022-04-20 08:03:18,SWAP,"CYNTHIA JOCABET OLAN RODRIGUEZ,,861.46,0020457...",SWAP 2022-04-20.csv,002045701331339825,002,2022-04-20,40002,BANAMEX


In [8]:
query_final.shape

(79203, 20)

# PAGOS DUPLICADOS

In [9]:
duplicates_query = query_final[query_final.duplicated(['report_id'],keep=False)]
duplicates_query.head()

,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,status,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,disbursement_method,swap,file_name,CLABE,BIN,date_disbursement,Clave de la institución,Nombre de la institución


In [10]:
duplicates_query.shape

(0, 20)

# VALIDACION MONTO A DISPERSAR

In [11]:
valid_1 = query.groupby(['date_disbursement']).agg({'amount':sum,'report_id':'count'}).reset_index()
valid_1.rename(columns={'report_id':'payments_number'},inplace=True)
valid_1['valor_comision'] = 1.85
valid_1['valor_iva'] = valid_1['valor_comision']*0.1600
valid_1['comision_dsp'] = valid_1['payments_number'] * valid_1['valor_comision']
valid_1['iva_dsp'] = valid_1['comision_dsp'] * 0.1600
valid_1['comision + iva'] = valid_1['comision_dsp'] + valid_1['iva_dsp']
valid_1['total_fondeo'] = valid_1['amount'] + valid_1['comision + iva']
valid_1['monto_mail'] = monto_me_mail
valid_1['comision_mail'] = comision_swap_mail
valid_1['iva_mail'] = iva_swap_mail
valid_1['monto_fondeo_mail'] = monto_fondear_mail
valid_1['check_monto'] = valid_1['amount'] - valid_1['monto_mail']
valid_1['check_comision'] = valid_1['comision_dsp'] - valid_1['comision_mail']
valid_1['check_iva'] = valid_1['iva_dsp'] - valid_1['iva_mail']
valid_1['check_fondeo'] = valid_1['total_fondeo'] - valid_1['monto_fondeo_mail']
valid_1['dsp'] = 'SWAP'
valid_1.head()

,date_disbursement,amount,payments_number,valor_comision,valor_iva,comision_dsp,iva_dsp,comision + iva,total_fondeo,monto_mail,comision_mail,iva_mail,monto_fondeo_mail,check_monto,check_comision,check_iva,check_fondeo,dsp
0,2022-04-20,"148,638,292.13",79203,1.85,0.30,"146,525.55","23,444.09","169,969.64","148,808,261.77","148,638,292.13","146,525.55","23,444.09","148,808,261.77",0.00,0.00,-0.00,-0.00,SWAP


In [12]:
valid_2 = valid_1[['date_disbursement','dsp','payments_number','amount','comision_dsp','iva_dsp','comision + iva','total_fondeo']]
valid_2.head()

,date_disbursement,dsp,payments_number,amount,comision_dsp,iva_dsp,comision + iva,total_fondeo
0,2022-04-20,SWAP,79203,"148,638,292.13","146,525.55","23,444.09","169,969.64","148,808,261.77"


In [13]:
valid_montos_final = valid_2.stack().reset_index(1)
valid_montos_final.rename(columns={'level_1':'concept',0:'data'},inplace=True)
valid_montos_final

,concept,data
0,date_disbursement,2022-04-20 00:00:00
0,dsp,SWAP
0,payments_number,79203
0,amount,"148,638,292.13"
0,comision_dsp,"146,525.55"
0,iva_dsp,"23,444.09"
0,comision + iva,"169,969.64"
0,total_fondeo,"148,808,261.77"


# EXCEL

In [14]:
writer = pd.ExcelWriter(r'G:\My Drive\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\04. APRIL\VALIDATION\VALIDATION 2022-04-20.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
query.to_excel(writer, sheet_name='DATA QUERY',index=False)
valid_montos_final.to_excel(writer, sheet_name='MONTOS',index=False)
valid_1.to_excel(writer, sheet_name='DIFF MONTOS',index=False)
duplicates_query.to_excel(writer, sheet_name='PAGOS DUPLICADOS',index=False)
writer.close()

In [15]:
valid_1.to_csv(r'G:\My Drive\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\04. APRIL\MONTOS EMAIL\DIFF MONTOS 2022-04-20.csv',index=False)

In [16]:
valid_2.to_csv(r'G:\My Drive\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\04. APRIL\VALIDATION\total_disbursement 2022-04-20.csv',index=False)

In [17]:
duplicates_query.to_csv(r'G:\My Drive\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\04. APRIL\VALIDATION\Duplicados 2022-04-20.csv',index=False)